# Assignment 4 / Text Summarization

In [1]:
import numpy as np
import pandas as pd
import string
import nltk
from nltk.tokenize.punkt import PunktSentenceTokenizer
from nltk.tokenize import sent_tokenize
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import networkx as nx

In [2]:
train_data = pd.read_csv("Summarization_train.csv", sep=',')
train_data = train_data.head(10000)
train_data.head()

,source,summary
0,Minghella war der Sohn italienisch-schottische...,"Anthony Minghella, CBE war ein britischer Film..."
1,Ende der 1940er Jahre wurde eine erste Auteur-...,Die Auteur-Theorie ist eine Filmtheorie und di...
2,"Al Pacino, geboren in Manhattan, ist der Sohn ...","Alfredo James ""Al"" Pacino ist ein US-amerikani..."
3,Der Name der Alkalimetalle leitet sich von dem...,Als Alkalimetalle werden die chemischen Elemen...
4,Die Arbeit ist bereits seit dem Altertum Gegen...,Das deutsche Arbeitsrecht ist ein Rechtsgebiet...


## Extractive summary creation

### Textrank
Als Hilfestellung hierfür habe ich folgendes Tutorial durchgearbeitet: https://joshbohde.com/blog/document-summarization/

Für den Textrank habe ich die Methode mit dem Graphen genommen mit folgenden Schritten:
1. Als erstes werden Texte in Sätze tokenisiert.
2. Sätze in Wortsammlungen tokenisieren, dargestellt als Vektor, wobei jede Dimension jeweils ein Wort räpresentiert. Die resultierende Matrix besteht aus Reihen von Sätzen und Kolonnen von Wörtern.
3. Damit die Matrix in einen Graphen umgewandelt werden kann, wird sie mithilfe des Tfidf-Transformers normalisiert, um die Wörter, die in jedem Satz vorkommen weniger zu gewichten.
4. Die Matrix wird in einen Graphen umgewandelt, indem sie mit ihrer transponierten Version multipliziert wird. Der daraus resultierende Graph (gespiegelte Matrix) stellt die Gleichheit zwischen den Sätzen dar
5. Abschliessend folgt das Bewerten und sortieren mittels Pagerank von NetworkX.

In [3]:
def textrank(document):
    sentence_tokenizer = PunktSentenceTokenizer()
    sentences = sentence_tokenizer.tokenize(document)
    vector_matrix = CountVectorizer().fit_transform(sentences)
    normalized_matrix = TfidfTransformer().fit_transform(vector_matrix)
    similarity_graph = normalized_matrix * normalized_matrix.T
    nx_graph = nx.from_scipy_sparse_matrix(similarity_graph)
    ranks = nx.pagerank(nx_graph)
    return sorted(((ranks[i],s) for i,s in enumerate(sentences)),
                  reverse=True)

### Summaries erstellen
Für jeden Text aus den eingespiesenen Daten, wird eine eine Zusammenfassung zwischen 85 und 115 Wörter aus bestehenden Sätzen extrahiert mit folgenden Schritten:

1. Der Satz mit dem höchsten Rang wird extrahiert und der Zusammenfassung hinzugefügt.
2. Der extrahierte Satz wird aus dem Text gelöscht, damit am Ende nicht zu viel redundante Information vorhanden ist.
3. Der Textrank wird wieder neu berechnet (ohne dem extrahierten Satz). Schritte 1 und 2 werden solange die Zusammenfassung weniger als 85 Wörter enthält wiederholt.
4. Wird mit dem Hinzufügen eines Satzes die Maximallänge (115 Wörter) überschritten, wird dieser Satz nicht hinzugefügt und aus dem Text gelöscht, bis ein Satz gefunden wird, mit dem eine gültige anzahl Wörter erreicht werden kann.

In [4]:
summaries = []
min_len = 85
max_len = 115
for i in range(len(train_data['source'])):
    summary = []
    length_summary = 0
    while length_summary < min_len:
        ranked_text = textrank(train_data['source'][i])
        if((length_summary + len(ranked_text[0][1].split())) > max_len):
            train_data['source'][i] = train_data['source'][i].replace(ranked_text[0][1], '')
        else:
            length_summary = length_summary + len(ranked_text[0][1].split())
            summary.append(ranked_text[0][1])
            train_data['source'][i] = train_data['source'][i].replace(ranked_text[0][1], '')  
    summaries.append(summary)

Hier den Index ändern um entsprechende Zusammenfassungen auszugeben:

In [5]:
print(summaries[0])

['Auch als Produzent war er erfolgreich, darunter für die Filme "Der stille Amerikaner", "Die Dolmetscherin" und "Der Vorleser", für den er 2008 posthum für den Oscar nominiert wurde.', 'Der Ehe entstammen zwei Kinder, die in der Filmbranche tätig sind: Tochter Hannah Minghella in der Produktion und Sohn Max Minghella als Schauspieler .', '1997 erhielt er für "Der englische Patient" den Oscar in der Rubrik "Beste Regie", 1999 eine Oscar-Nominierung in der Kategorie "Bestes adaptiertes Drehbuch" für "Der talentierte Mr.', 'Minghella war der Sohn italienisch-schottischer Eltern, die auf der Isle of Wight eine Fabrik für Eiscreme betrieben.']


## Evaluation of summaries

### Preparing Data
Um den Rouge-Score berechnen zu können bilde ich aus den Zusammenfassungen Listen aus Uni- und Bigrammen. In den Folien (NLP_Woche_11.ppt Folie 32) werden die Anzahl n-Gramme jeweils als mathematische Mengen in den Formeln angegeben, daher entferne ich alle duplikate in den Listen von n-Grammen.

In [6]:
def createListOfUnigrams(summary):
    summary = summary.translate(str.maketrans('', '', string.punctuation)).lower()
    summary_words = summary.split()
    return list(dict.fromkeys(summary_words))

In [7]:
def createListOfBigrams(summary_words):
    summary_tuples = []
    for i in range(len(summary_words)-1):
        list_tuple = [summary_words[i], summary_words[i+1]]
        summary_tuples.append(tuple(list_tuple))
    return summary_tuples

In [8]:
def createListsOfUniBiSum(list_summaries):
    list_summaries_unigrams = []
    list_summaries_bigrams = []
    for s in list_summaries:
        list_summaries_unigrams.append(createListOfUnigrams(s))
    for u in list_summaries_unigrams:
        list_summaries_bigrams.append(createListOfBigrams(u))
    return list_summaries_unigrams, list_summaries_bigrams

In [9]:
created_summaries = []
reference_summaries = train_data['summary']

for summary in summaries:
    text = ""
    for sentence in summary:
        text = text + sentence
    created_summaries.append(text)

created_summaries_unigrams, created_summaries_bigrams = createListsOfUniBiSum(created_summaries)
reference_summaries_unigrams, reference_summaries_bigrams = createListsOfUniBiSum(reference_summaries)

print(created_summaries_unigrams[0])
print()
print(created_summaries_bigrams[0])
print()
print(reference_summaries_unigrams[0])
print()
print(reference_summaries_bigrams[0])

['auch', 'als', 'produzent', 'war', 'er', 'erfolgreich', 'darunter', 'für', 'die', 'filme', 'der', 'stille', 'amerikaner', 'dolmetscherin', 'und', 'vorleser', 'den', '2008', 'posthum', 'oscar', 'nominiert', 'wurdeder', 'ehe', 'entstammen', 'zwei', 'kinder', 'in', 'filmbranche', 'tätig', 'sind', 'tochter', 'hannah', 'minghella', 'produktion', 'sohn', 'max', 'schauspieler', '1997', 'erhielt', 'englische', 'patient', 'rubrik', 'beste', 'regie', '1999', 'eine', 'oscarnominierung', 'kategorie', 'bestes', 'adaptiertes', 'drehbuch', 'talentierte', 'mrminghella', 'italienischschottischer', 'eltern', 'auf', 'isle', 'of', 'wight', 'fabrik', 'eiscreme', 'betrieben']

[('auch', 'als'), ('als', 'produzent'), ('produzent', 'war'), ('war', 'er'), ('er', 'erfolgreich'), ('erfolgreich', 'darunter'), ('darunter', 'für'), ('für', 'die'), ('die', 'filme'), ('filme', 'der'), ('der', 'stille'), ('stille', 'amerikaner'), ('amerikaner', 'dolmetscherin'), ('dolmetscherin', 'und'), ('und', 'vorleser'), ('vo

### Rouge-n Scores 
Für die Berechnung des Rouge-Scores wird in einem ersten Schritt Recall und Precision gemäss den Formeln in den Folien (NLP_Woche_11.ppt Folie 32) berechnet. Die Funktion habe ich so definiert, dass sie für beliebige Rouge-n-Scores verwendet werden kann. Im Unterricht der Woche 11 wurde empfohlen, dass man anstelle von alpha = 0.5 (Folie 33) auch den F1-Score verwenden kann/darf. Ich habe dann die Funktion mit beiden Werten (0.5 und F1) getestet und mit dem F1 bessere Rouge-Scores  (vor allem bei Rouge-2) erhalten, weshalb ich dann diese herangehensweise gewählt habe.
Als ich dann die Funktion über alle Zusammenfassungen laufen liess, hat sich herausgestellt dass einzelne wenige einen Recall und Precision von 0 hatten, daher gebe ich direkt einen Rouge-Score von 0 zurück um einen Divide by 0 error zu verhindern, sollte dies der Fall sein.
Abschliessend wird noch der durchschnittliche Rouge-Score über alle Zusammenfassungen berechnet und ausgegeben.
Da die Laufzeit extrem lange wird bei mehr Texten, erstelle ich Zusammenfassungen nur aus den ersten 10000.

In [10]:
def calcRouge(created_summary_words, reference_summary_words):
    sum_ref = len(reference_summary_words)
    sum_crtd = len(created_summary_words)
    sum_crtd_ref = 0
    for w in created_summary_words:
        if(w in reference_summary_words):
            sum_crtd_ref = sum_crtd_ref + 1
    r_rouge = sum_crtd_ref / sum_ref
    p_rouge = sum_crtd_ref / sum_crtd
    if((r_rouge == 0.0) & (p_rouge == 0.0)):
        return r_rouge, p_rouge, 0, 0
    else:
        f1 = 2 * (p_rouge * r_rouge) / (p_rouge + r_rouge)
        rouge = (r_rouge * p_rouge) / ((f1 * r_rouge) + (1-f1) * (p_rouge))
        return r_rouge, p_rouge, f1, rouge

In [11]:
def averageRouge(created_summary_words, reference_summary_words):
    recall_rouge_1 = 0
    precision_rouge_1 = 0
    f1_rouge_1 = 0
    rouge_1 = 0
    for i in range(len(created_summary_words)):
        rc_1, pr_1, f1_1, r_1 = calcRouge(created_summary_words[i], reference_summary_words[i])
        recall_rouge_1 = recall_rouge_1 + rc_1 
        precision_rouge_1 = precision_rouge_1 + pr_1
        f1_rouge_1 = f1_rouge_1 + f1_1
        rouge_1 = rouge_1 + r_1
    recall_rouge_1 = recall_rouge_1 / len(created_summary_words)
    precision_rouge_1 = precision_rouge_1 / len(created_summary_words)
    f1_rouge_1 = f1_rouge_1 / len(created_summary_words)
    rouge_1 = rouge_1 / len(created_summary_words)
    return recall_rouge_1, precision_rouge_1, f1_rouge_1, rouge_1

### Ausgabe Rouge-1

In [12]:
recall_rouge_1, precision_rouge_1, f1_rouge_1, rouge_1 = averageRouge(created_summaries_unigrams, reference_summaries_unigrams)
print("recall: ", recall_rouge_1)
print("precision: ", precision_rouge_1)
print("f1: ", f1_rouge_1)
print("rouge_1: ", rouge_1)

recall:  0.36199436754721404
precision:  0.15742786696838096
f1:  0.2074740227995985
rouge_1:  0.27194055503505377


### Ausgabe Rouge-2

In [13]:
recall_rouge_2, precision_rouge_2, f1_rouge_2, rouge_2 = averageRouge(created_summaries_bigrams, reference_summaries_bigrams)
print("recall: ", recall_rouge_2)
print("precision: ", precision_rouge_2)
print("f1: ", f1_rouge_2)
print("rouge_2: ", rouge_2)

recall:  0.03927008306845611
precision:  0.015702284177859346
f1:  0.02113519063846897
rouge_2:  0.03489116789365048
